## 모듈 가져오기

In [1]:
from jetbotmini import Camera
from jetbotmini import bgr8_to_jpeg

import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

import os

## 카메라 객체 생성

In [2]:
camera = Camera.instance(width=224, height=224)

## 카메라 영상을 출력할 위젯 생성

In [3]:
image = widgets.Image(format='jpeg', width=224, height=224)

## 카메라 영상이 변경되면 위젯에 표시할 이미지도 변경되도록 링크

In [4]:
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

## 위젯 출력

In [5]:
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## 학습 이미지를 저장할 디렉토리를 레이블 별로 생성
* 아래 셀 실행 후 좌측 파일 탐색기에 dataset/blocked 디렉토리와 dataset/free 디렉토리가 생성됨을 확인할 수 있음

In [6]:
try:
    os.makedirs('dataset/free')
    os.makedirs('dataset/blocked')
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


## 학습 데이터 수집(이미지 저장)을 위한 버튼 위젯 생성

In [7]:
# 이미지 저장 버튼
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)

# 저장된 이미지 수를 출력할 텍스트 박스
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir('dataset/free')))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir('dataset/blocked')))

# 버튼 및 텍스트 박스 출력
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

## 버튼 클릭시 이미지를 디렉토리에 저장하도록 구현

In [8]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())